# Singapore Real-time Weather (NEA + Plotly)

This notebook reuses the data-wrangling ideas from `weather_singapore.ipynb` and focuses on a Plotly-only visualisation that highlights the most recent air-temperature readings published by Singapore's National Environment Agency (NEA).


In [1]:
import requests
import pandas as pd
import plotly.graph_objects as go
from zoneinfo import ZoneInfo


In [2]:
BASE_URL = "https://api-open.data.gov.sg/v2/real-time/api"
AIR_TEMPERATURE_ENDPOINT = "air-temperature"
SINGAPORE_TZ = ZoneInfo("Asia/Singapore")


In [3]:
def fetch_realtime_air_temperature() -> dict:
    """Fetch the latest NEA air-temperature readings."""
    url = f"{BASE_URL}/{AIR_TEMPERATURE_ENDPOINT}"
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    return response.json()


data = fetch_realtime_air_temperature()
data


{'code': 0,
 'data': {'stations': [{'id': 'S109',
    'deviceId': 'S109',
    'name': 'Ang Mo Kio Avenue 5',
    'location': {'latitude': 1.3764, 'longitude': 103.8492}},
   {'id': 'S106',
    'deviceId': 'S106',
    'name': 'Pulau Ubin',
    'location': {'latitude': 1.4168, 'longitude': 103.9673}},
   {'id': 'S117',
    'deviceId': 'S117',
    'name': 'Banyan Road',
    'location': {'latitude': 1.256, 'longitude': 103.679}},
   {'id': 'S107',
    'deviceId': 'S107',
    'name': 'East Coast Parkway',
    'location': {'latitude': 1.3135, 'longitude': 103.9625}},
   {'id': 'S115',
    'deviceId': 'S115',
    'name': 'Tuas South Avenue 3',
    'location': {'latitude': 1.29377, 'longitude': 103.61843}},
   {'id': 'S102',
    'deviceId': 'S102',
    'name': 'Semakau Landfill',
    'location': {'latitude': 1.189, 'longitude': 103.768}},
   {'id': 'S60',
    'deviceId': 'S60',
    'name': 'Sentosa',
    'location': {'latitude': 1.25, 'longitude': 103.8279}},
   {'id': 'S50',
    'deviceId': '

In [4]:
stations_df = pd.json_normalize(data["data"]["stations"])
readings_block = data["data"]["readings"][0]
readings_df = pd.json_normalize(readings_block["data"])

timestamp = pd.to_datetime(readings_block["timestamp"], utc=True).tz_convert(SINGAPORE_TZ)

weather_df = (
    stations_df
    .merge(readings_df, left_on="id", right_on="stationId")
    .rename(
        columns={
            "location.latitude": "latitude",
            "location.longitude": "longitude",
            "value": "temperature_celsius",
        }
    )
)

weather_df.head()


,id,deviceId,name,latitude,longitude,stationId,temperature_celsius
0,S109,S109,Ang Mo Kio Avenue 5,1.37640,103.84920,S109,31.4
1,S106,S106,Pulau Ubin,1.41680,103.96730,S106,30.8
2,S117,S117,Banyan Road,1.25600,103.67900,S117,30.1
3,S107,S107,East Coast Parkway,1.31350,103.96250,S107,30.2
4,S115,S115,Tuas South Avenue 3,1.29377,103.61843,S115,31.0


In [12]:
center_lat = weather_df["latitude"].mean()
center_lon = weather_df["longitude"].mean()

hover_text = weather_df.apply(
    lambda row: f"{row['name']}<br>{row['temperature_celsius']:.1f}°C",
    axis=1,
)

fig = go.Figure()

fig.add_trace(
    go.Densitymapbox(
        lat=weather_df["latitude"],
        lon=weather_df["longitude"],
        z=weather_df["temperature_celsius"],
        radius=40,
        # color_continuous_scale="Turbo",
        colorbar=dict(title="Celsius"),
        # hoverinfo="skip",
        # hoverinfo="skip"
    )
)

fig.add_trace(
    go.Scattermapbox(
        lat=weather_df["latitude"],
        lon=weather_df["longitude"],
        text=hover_text,
        mode="markers+text",
        marker=dict(
            size=12,
            color="rgba(0,0,0,0)",
            line=dict(width=3, color="white"),
        ),
        textposition="top center",
        textfont=dict(size=10, color="#111"),
        hoverinfo="text",
    )
)

fig.update_layout(
    title=f"NEA real-time air temperature • {timestamp.strftime('%d %b %Y %H:%M %Z')}",
    mapbox=dict(
        style="carto-positron",
        center=dict(lat=center_lat, lon=center_lon),
        zoom=11.2,
    ),
    margin=dict(l=10, r=10, t=60, b=10),
)

fig.show()


C:\Users\00492\AppData\Local\Temp\ipykernel_33552\53275980.py:12: DeprecationWarning:

*densitymapbox* is deprecated! Use *densitymap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



ValueError: Invalid property specified for object of type plotly.graph_objs.scattermapbox.Marker: 'line'

Did you mean "size"?

    Valid properties:
        allowoverlap
            Flag to draw all symbols, even if they overlap.
        angle
            Sets the marker orientation from true North, in degrees
            clockwise. When using the "auto" default, no rotation
            would be applied in perspective views which is
            different from using a zero angle.
        anglesrc
            Sets the source reference on Chart Studio Cloud for
            `angle`.
        autocolorscale
            Determines whether the colorscale is a default palette
            (`autocolorscale: true`) or the palette determined by
            `marker.colorscale`. Has an effect only if in
            `marker.color` is set to a numerical array. In case
            `colorscale` is unspecified or `autocolorscale` is
            true, the default palette will be chosen according to
      